In [31]:
from selenium import webdriver

In [32]:
import datetime, re, requests, io, time, random, string

In [33]:
from bs4 import BeautifulSoup

In [34]:
from credentials import email, password

In [35]:
driver = webdriver.Chrome('/Users/yoel/downloads/chromedriver')

In [36]:
# Go to wallmine.com
driver.get('https://wallmine.com')

In [37]:
if 'Make Smarter Investments' in driver.page_source:
    sign_in_link = driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a')
    sign_in_link.click()
    time.sleep(3)
    print('Going to sign in page')

Going to sign in page


In [38]:
if 'Sign in here, please.' in driver.page_source:
    login_email = driver.find_element_by_xpath('//*[@id="user_email"]')
    sign_in_password = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]')
    sign_in_password.click()
    time.sleep(0.2)

In [39]:
login_password = driver.find_element_by_xpath('//*[@id="user_password"]')
sign_in_button = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button')

In [40]:
login_email.send_keys(email)
login_password.send_keys(password)
sign_in_button.click()
time.sleep(3)

In [42]:
if 'Stock market overview' in driver.page_source:
    print('On the main page')
    heatmap = driver.find_element_by_xpath('//*[@id="homepage-heatmap"]/a/div[2]')
    heatmap.click()
    time.sleep(2)

On the main page


In [44]:
overview_tab = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a')
overview_tab.click()
time.sleep(3)

In [50]:
if 'Free Stock Screener' in driver.page_source:
    print('here')
    stock_data = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]').text
stock_list = stock_data.split('\n')
stock_list

here


['▴ ▴',
 'Symbol',
 '▴ ▴',
 'Company',
 '▴ ▴',
 'Exchange',
 '▴ ▴',
 'Industry',
 '▴ ▴',
 'Market cap',
 '▴ ▴',
 'EBITDA',
 '▴ ▴',
 'P/E',
 '▴ ▴',
 'EV/EBITDA',
 '▴ ▴',
 'Debt/Equity',
 '▴ ▴',
 'Average volume',
 '▴ ▴',
 'Institutional ownership',
 '▴ ▴',
 'Earnings date',
 '▴ ▴',
 'Price',
 '▴ ▴',
 'Performance today',
 'AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 33.68 24.95 2.74 52.096M 62.12% 29 Oct 2020 $109.17 -2.36%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 35.64 22.40 1.55 34.993M 74.09% 22 Oct 2020 $204.12 -1.59%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.51T $30.457B 116.75 49.32 2.63 5.062M 57.69% 22 Oct 2020 $3,053.36 -2.42%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 29.55 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,434.20 -2.13%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 31.85 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,427.05 -2.24%',


In [51]:
stock_list.index('Performance today')

27

In [54]:
stock_list[28]

'AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 33.68 24.95 2.74 52.096M 62.12% 29 Oct 2020 $109.17 -2.36%'

In [67]:
stock_list[28].split(' $')
company_info_parsed = stock_list[28].split(' $')[0]
market_cap = stock_list[28].split(' $')[1]
market_info = stock_list[28].split(' $')[2]
price_info = stock_list[28].split(' $')[3].split(' ')
price = f'${price_info[0]}'
performance_today = price_info[1]
# print(company_info_parsed)
# price_info
# price

In [78]:
company_info = company_info_parsed.split(' ')
if 'NASDAQ' in company_info:
    idx = company_info.index('NASDAQ')
    exchange = company_info[idx]
    print('here')
elif 'NYSE' in company_info:
    idx = company_info.index('NYSE')
    exchange = company_info[idx]
elif 'NYSEMKT' in company_info:
    idx = company_info.index('NYSEMKT')
    exchange = company_info[idx]
    
industry = " ".join(company_info[idx + 1:])
symbol = company_info[0]
company_name = " ".join(company_info[1:idx])
company_name

# Currently parsed
# symbol, company_name, exchange, industry, market_cap, price, performance_today
# ebitda, p_e, ev_ebitda, debt_equity, average_volume, institutional_ownership, earnings_date

here


'Apple, Inc.'

In [80]:
# Parse market info
market_info.split(' ')

['74.542B', '33.68', '24.95', '2.74', '52.096M', '62.12%', '29', 'Oct', '2020']

In [81]:
ebitda = market_info.split(' ')[0]
p_e = market_info.split(' ')[1]
ev_ebitda = market_info.split(' ')[2]
debt_equity = market_info.split(' ')[3]
average_volume = market_info.split(' ')[4]
institutional_ownership = market_info.split(' ')[5]
earnings_date = " ".join(market_info.split(' ')[6:])
earnings_date

'29 Oct 2020'

In [82]:
x_stock = {
    "symbol": symbol,
    "company_name": company_name,
    "exchange": exchange,
    "industry": industry,
    "market_cap": market_cap,
    "price": price,
    "performance_today":  performance_today,
    "ebitda": ebitda,
    "p_e": p_e,
    "ev_ebitda": ev_ebitda,
    "debt_equity": debt_equity,
    "average_volume": average_volume,
    "institutional_ownership": institutional_ownership,
    "earnings_date": earnings_date
}

x_stock

{'symbol': 'AAPL',
 'company_name': 'Apple, Inc.',
 'exchange': 'NASDAQ',
 'industry': 'Consumer Electronics',
 'market_cap': '1.91T',
 'price': '$109.17',
 'performance_today': '-2.36%',
 'ebitda': '74.542B',
 'p_e': '33.68',
 'ev_ebitda': '24.95',
 'debt_equity': '2.74',
 'average_volume': '52.096M',
 'institutional_ownership': '62.12%',
 'earnings_date': '29 Oct 2020'}

In [87]:
def parse_stock_data(data):
    company_info_parsed = data.split(' $')[0]
    market_cap = data.split(' $')[1]
    market_info = data.split(' $')[2]
    price_info = data.split(' $')[3].split(' ')
    price = f'${price_info[0]}'
    performance_today = price_info[1]
    company_info = company_info_parsed.split(' ')
    
    if 'NASDAQ' in company_info:
        idx = company_info.index('NASDAQ')
        exchange = company_info[idx]
        print('here')
    elif 'NYSE' in company_info:
        idx = company_info.index('NYSE')
        exchange = company_info[idx]
    elif 'NYSEMKT' in company_info:
        idx = company_info.index('NYSEMKT')
        exchange = company_info[idx]

    industry = " ".join(company_info[idx + 1:])
    symbol = company_info[0]
    company_name = " ".join(company_info[1:idx])
    ebitda = market_info.split(' ')[0]
    p_e = market_info.split(' ')[1]
    ev_ebitda = market_info.split(' ')[2]
    debt_equity = market_info.split(' ')[3]
    average_volume = market_info.split(' ')[4]
    institutional_ownership = market_info.split(' ')[5]
    earnings_date = " ".join(market_info.split(' ')[6:])
    
    result = {
        "symbol": symbol,
        "company_name": company_name,
        "exchange": exchange,
        "industry": industry,
        "market_cap": market_cap,
        "price": price,
        "performance_today":  performance_today,
        "ebitda": ebitda,
        "p_e": p_e,
        "ev_ebitda": ev_ebitda,
        "debt_equity": debt_equity,
        "average_volume": average_volume,
        "institutional_ownership": institutional_ownership,
        "earnings_date": earnings_date
    }
    
    return result
    

In [88]:
parse_stock_data('AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 33.68 24.95 2.74 52.096M 62.12% 29 Oct 2020 $109.17 -2.36%')

here


{'symbol': 'AAPL',
 'company_name': 'Apple, Inc.',
 'exchange': 'NASDAQ',
 'industry': 'Consumer Electronics',
 'market_cap': '1.91T',
 'price': '$109.17',
 'performance_today': '-2.36%',
 'ebitda': '74.542B',
 'p_e': '33.68',
 'ev_ebitda': '24.95',
 'debt_equity': '2.74',
 'average_volume': '52.096M',
 'institutional_ownership': '62.12%',
 'earnings_date': '29 Oct 2020'}

In [94]:
all_stocks = stock_list[28:]
all_stocks.index('BABA Alibaba Group Holding Ltd. NYSE Specialty Retail $747.81B ¥19.774B 28.63 242.16 0.57 15.474M 48.80% 30 Oct 2020 $274.46 -0.30%')
baba = all_stocks.pop(5)

In [95]:
for each_stock in all_stocks:
    print(parse_stock_data(each_stock))

here
{'symbol': 'AAPL', 'company_name': 'Apple, Inc.', 'exchange': 'NASDAQ', 'industry': 'Consumer Electronics', 'market_cap': '1.91T', 'price': '$109.17', 'performance_today': '-2.36%', 'ebitda': '74.542B', 'p_e': '33.68', 'ev_ebitda': '24.95', 'debt_equity': '2.74', 'average_volume': '52.096M', 'institutional_ownership': '62.12%', 'earnings_date': '29 Oct 2020'}
here
{'symbol': 'MSFT', 'company_name': 'Microsoft Corp.', 'exchange': 'NASDAQ', 'industry': 'Internet Services & Infrastructure', 'market_cap': '1.54T', 'price': '$204.12', 'performance_today': '-1.59%', 'ebitda': '65.259B', 'p_e': '35.64', 'ev_ebitda': '22.40', 'debt_equity': '1.55', 'average_volume': '34.993M', 'institutional_ownership': '74.09%', 'earnings_date': '22 Oct 2020'}
here
{'symbol': 'AMZN', 'company_name': 'Amazon.com, Inc.', 'exchange': 'NASDAQ', 'industry': 'Specialty Retail', 'market_cap': '1.51T', 'price': '$3,053.36', 'performance_today': '-2.42%', 'ebitda': '30.457B', 'p_e': '116.75', 'ev_ebitda': '49.32'

IndexError: list index out of range

In [96]:
all_stocks

['AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 33.68 24.95 2.74 52.096M 62.12% 29 Oct 2020 $109.17 -2.36%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 35.64 22.40 1.55 34.993M 74.09% 22 Oct 2020 $204.12 -1.59%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.51T $30.457B 116.75 49.32 2.63 5.062M 57.69% 22 Oct 2020 $3,053.36 -2.42%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 29.55 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,434.20 -2.13%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 31.85 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,427.05 -2.24%',
 'FB Facebook, Inc. NASDAQ Internet Services & Infrastructure $725.94B $34.727B 30.88 19.46 0.32 26.764M 79.41% 28 Oct 2020 $252.81 -0.76%',
 'BRK-A Berkshire Hathaway Inc. NYSE Multi-line Insurance $520.31B $53.661B 19.54 11.11 0.92 1.012k 24.67% N/A $318,812.00 -0.56%',
 'BRK-B Berkshire Hathaway Inc. NYSE Multi-line I